**Building an AI Agent from Scratch using LangChain**

Installing the needed packages

In [1]:
!pip install --quiet -U langchain langchain_openai tavily-python
# !pip install -U tavily-python
!pip install -U langchain-community
# !pip install -U langchain langchain-openai langchain-tavily
!pip install --upgrade langgraph

Setting API keys for OpenAI (LLM being used) and Tavily (Search tool being used)

In [2]:
!pip install python-dotenv

In [3]:
# #To input the keys manualyy
# import os
# import getpass

# os.environ["OPEN_API_KEY"] = getpass.getpass("OpenAI API Key:")
# os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

# To load the keys from the env created
from dotenv import load_dotenv
import os

load_dotenv()

# Correct variable names
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")
langchain_key=os.getenv("LANGCHAIN_API_KEY")

# Set them as environment variables explicitly (for LangChain/OpenAI to pick them up)
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["TAVILY_API_KEY"] = tavily_key
os.environ["LANGCHAIN_API_KEY"]=langchain_key

# Confirm
print("✅ OpenAI key loaded:", bool(openai_key))
print("✅ Tavily key loaded:", bool(tavily_key))
print("✅ Langchain key loaded:", bool(langchain_key))

✅ OpenAI key loaded: True
✅ Tavily key loaded: True
✅ Langchain key loaded: True


**Now let's create the LangChain AI Agents**

In [4]:
!pip show langgraph

Name: langgraph
Version: 0.5.4
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License-Expression: MIT
Location: c:\users\akino\anaconda3\envs\nlp_project\lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: 


In [5]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools =[TavilySearchResults(max_results=10)]

#Prompt type used
prompt= hub.pull("hwchase17/openai-functions-agent")

#Choose the LLM that will drive the agent
# llm= ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True
llm= ChatOpenAI(model="gpt-4", streaming=True)               

#OpenAI Function Agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

C:\Users\akino\AppData\Local\Temp\ipykernel_1980\4009537192.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tools =[TavilySearchResults(max_results=10)]


**Now let's define the graph state**

input-->chat_history---->intermediate_steps---->agent_outcome

In [6]:
from typing import TypedDict, Annotated, List, Union, Tuple
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    #Input string
    input:str
    # Previous message list in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs 'None' as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # Actions and Observations
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

**Defining the nodes**

In [7]:
from langchain_core.agents import AgentFinish, AgentAction

# Custom tool executor
def custom_tool_executor(agent_action, tools):
    for tool in tools:
        if tool.name == agent_action.tool:
            return tool.invoke(agent_action.tool_input)
    raise ValueError(f"Tool '{agent_action.tool}' not found.")

# Agent definition
def run_agent (data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Function execution tool
def execute_tools(data):
    # most recent agent_outcome 
    agent_action = data['agent_outcome']
    output = custom_tool_executor(agent_action, tools)
    return{"intermediate_steps": [(agent_action, str(output))]}

#conditional edge logic
def should_continue(data):
    # FOR AgentFinish, return 'exit' string
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    
    else:
        return "continue"

**Defining the graph**

In [8]:
from langgraph.graph import END, StateGraph

#Defining a new graph
workflow=StateGraph(AgentState)

#Two nodes to iterate within
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

#Entry point as an agent
workflow.set_entry_point("agent")

#We now add a conditional edge
workflow.add_conditional_edges(
    # Defining the start node as an agent
    "agent",
    # Determine which node is next
    should_continue,
    # Continue or end
    {
        "continue":"action",
        # else, finish
        "end":END
    }
)

#adding an edge
workflow.add_edge('action', 'agent')

# Compiling
app=workflow.compile()    

In [9]:
inputs={"input":"Anything about The processdoctorgroup on linkedin", "chat_history":[]}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'The processdoctorgroup on linkedin'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'The processdoctorgroup on linkedin'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "The processdoctorgroup on linkedin"\n}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4-0613', 'service_tier': 'default'}, id='run--5184d75c-25fa-46b9-a398-b332d01dd314-0')])}
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'The processdoctorgroup on linkedin'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'The processdoctorgroup on linkedin'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "The processdoctorgroup on linkedin"\n}', 'name': 'tavil